In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
A220_tech_docs_text = dataiku.Folder("rhnW9xGx")
A220_tech_docs_text_info = A220_tech_docs_text.get_info()

# Assuming the folder contains text files, we can read them into a DataFrame
import os

In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd
import numpy as np
from dataiku import pandasutils as pdu
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

# Read recipe inputs
A220_tech_docs_text = dataiku.Folder("rhnW9xGx")
A220_tech_docs_text_info = A220_tech_docs_text.get_info()

# Assuming the folder contains text files, we can read them into a DataFrame
import os

file_paths = A220_tech_docs_text.list_paths_in_partition()
texts = []

for file_path in file_paths:
    with A220_tech_docs_text.get_download_stream(file_path) as f:
        texts.append(f.read().decode('utf-8'))

# Extract sentences from the texts
sentences = []
for text in texts:
    text = text.replace("\x0c", "\n")
    text = text.strip()
    sentences.extend(sent_tokenize(text))

# Create a DataFrame from the sentences
sentences_df = pd.DataFrame({'sentence': sentences})

# Splitting sentences by newline character
sentences_df['sentence'] = sentences_df['sentence'].apply(lambda x: x.split('\n'))
# Stripping whitespace from each sentence
exploded_df = sentences_df.explode('sentence')
exploded_df = exploded_df.drop_duplicates()
exploded_df['sentence'] = exploded_df['sentence'].apply(lambda x: x.strip() if isinstance(x, str) else x)
exploded_df['sentence'].replace('', np.nan, inplace=True)
exploded_df = exploded_df.dropna().reset_index(drop=True)
clean_sentences_df = exploded_df


In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd
import numpy as np
from dataiku import pandasutils as pdu
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Increase the maximum length limit
nlp.max_length = 3000000  # Adjust this value as needed

# Read recipe inputs
A220_tech_docs_text = dataiku.Folder("rhnW9xGx")
A220_tech_docs_text_info = A220_tech_docs_text.get_info()

# Assuming the folder contains text files, we can read them into a DataFrame
import os

file_paths = A220_tech_docs_text.list_paths_in_partition()
texts = []

for file_path in file_paths:
    with A220_tech_docs_text.get_download_stream(file_path) as f:
        texts.append(f.read().decode('utf-8'))

# Extract sentences from the texts using spaCy
sentences = []
for text in texts:
    text = text.replace("\x0c", "\n")
    text = text.strip()
    doc = nlp(text)
    sentences.extend([sent.text for sent in doc.sents])

# Create a DataFrame from the sentences
sentences_df = pd.DataFrame({'sentence': sentences})

# Splitting sentences by newline character
sentences_df['sentence'] = sentences_df['sentence'].apply(lambda x: x.split('\n'))

# Exploding the lists into separate rows
exploded_df = sentences_df.explode('sentence')

# Dropping duplicates
exploded_df = exploded_df.drop_duplicates()

# Stripping whitespace from each sentence
exploded_df['sentence'] = exploded_df['sentence'].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Dropping empty rows
exploded_df['sentence'].replace('', np.nan, inplace=True)
exploded_df = exploded_df.dropna().reset_index(drop=True)

clean_sentences_df = exploded_df

print(clean_sentences_df)


In [0]:
clean_sentences_df

In [0]:
# Write recipe outputs
sentences = dataiku.Dataset("sentences")
sentences.write_with_schema(sentences_df)

In [0]:
import spacy

# Install the spaCy model
import subprocess
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])

In [0]:
import subprocess

# Install spaCy
subprocess.run(["pip", "install", "spacy"])

# Install the spaCy model
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])

In [0]:
pip install spacy

In [0]:
import spacy
spacy.cli.download('en_core_web_sm')